In [10]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import pickle

In [2]:
df_anime = pd.read_csv('anime.csv', usecols=['anime_id', 'name'], dtype={'anime_id': 'int32', 'name': 'str'})
df_ratings = pd.read_csv('rating.csv', dtype={'anime_id': 'int32', 'user_id': 'int32', 'rating':'float32'})[['anime_id', 'user_id', 'rating']]


In [3]:
# Sparse Matrix: Creates a matrix with animes as the rows, users as columns, and ratings as the entries
# Creating table that'll be converted to sparse matrix; anime_id for rows and user_id for cols
anime_users = df_ratings.pivot_table(index='anime_id',columns='user_id',values='rating').fillna(0)
# convert dataframe of anime features to scipy sparse matrix
anime_user_matrix = csr_matrix(anime_users.values)

In [4]:
# Formulas for getting distance between the vectors: Euclidean Distance, Manhattan Distance, Minkowski Distance
# Cosine Similarity calculates the percentage of similarity
# Algorithms: Brute force, ball tree, and KD tree
NUM_RECOMMENDED = 20
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=NUM_RECOMMENDED)   

In [5]:
# Training the model
model_knn.fit(anime_user_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [6]:
def get_anime_id(name):
    anime_details = process.extractOne(name, df_anime['name'])
    # Ex: ('Naruto: Shippuden', 91, 615) gives the name, accuracy, and id number
    id = anime_details[2]
    return id

In [7]:
def get_anime_names(indices):
    names = [str(df_anime['name'][x]) for x in indices[0][1:]]
    return names


In [8]:
# Recommender(movie_name) => List of Recommended Anime
def recommender(anime_name, data_matrix, model, n_recommendations):
    model.fit(data_matrix)
    id = get_anime_id(anime_name)
    print(f"Suggested anime if you\'ve watched {df_anime['name'][id]}")
    distances, indices = model.kneighbors(data_matrix[id], n_neighbors=n_recommendations)
    recommended = get_anime_names(indices)
    print("\n".join(recommended))
recommender('Fullmetal Alchemist: Brotherhood', anime_user_matrix, model_knn, 20)

Suggested anime if you've watched Fullmetal Alchemist: Brotherhood
Kimi no Na wa.
One Punch Man
Mushishi Zoku Shou
Gintama°
Major: Message
Hunter x Hunter: Greed Island
Code Geass: Hangyaku no Lelouch
Jigoku Shoujo Futakomori
Nanatsu no Taizai
Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen
Junjou Romantica
Seikai no Monshou
Jin-Rou
City Hunter: The Secret Service
Katekyo Hitman Reborn!
Shuangsheng Lingtan
Break Blade 4: Sanka no Chi
One Piece: Nenmatsu Tokubetsu Kikaku! Mugiwara no Luffy Oyabun Torimonochou
Barakamon


In [12]:
with open('ratings_model_knn.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [ ]:
with open('ratings_matrix_knn.pkl', 'wb') as f:
    pickle.dump(anime_user_matrix, f)